# Model search

## Getting started

With training sets generated and hyperparameter sets designed, its time to search for a best performing model setup. Models will be evaluated at three levels: **1)** their ability to identify scenes with seals, **2)** retrieve seal haul outs **3)** and count individual seals. Current training classes can be displayed by running the following cells.

In [ ]:
import os
class_names = [x[1] for x in os.walk('./training_set/training')][0]
print(class_names)

In [ ]:
# The directories should match
%ls ./training_set/training
print('\n')
%ls ./training_set/validation

## Training

The first step to find a best performing model is to train different model setups using our training set. To keep track of which combinations we have tried and, later on, performance scores at the three levels described above, we will start by creating a pandas DataFrame with model definitions.

In [1]:
import pandas as pd

combinations = {'model_architecture': ['Resnet18' for ele in range(4)] + ['Nasnet1' for ele in range(4)],
                'training_dir': ['training_set', 'training_set_multiscale'] * 4,
                'hyperparameter_set': ['A' for ele in range(4)] + ['B' for ele in range(4)],
                'cv_weights': ['NO', 'NO', 'A', 'A'] * 2,
                'output_name': ['model{}.tar'.format(i) for i in range(1,9)]}

combinations = pd.DataFrame(combinations)



We can then provide model combinations created above as arguments to the training script, *train_sealnet.py*. A list of requiredarguments can be displayed by running the cell below.

In [2]:
%run train_sealnet.py -h

usage: train_sealnet.py [-h]
                        training_dir model_architecture hyperparameter_set
                        cv_weights output_name

trains a CNN to find seals in satellite imagery

positional arguments:
  training_dir        base directory to recursively search for images in
  model_architecture  model architecture, must be a member of models
                      dictionary
  hyperparameter_set  combination of hyperparameters used, must be a member of
                      hyperparameters dictionary
  cv_weights          weights for weighted-cross validation, must be a member
                      of cv_weightsdictionary
  output_name         name of output file from training, this name will also
                      be used in subsequent steps of the pipeline

optional arguments:
  -h, --help          show this help message and exit


In [3]:
for row in combinations.iterrows():
    t_dir, arch, hyp_st, cv_wgt, out = row[1]['training_dir'], row[1]['model_architecture'], row[1]['hyperparameter_set'], row[1]['cv_weights'], row[1]['output_name']                                    
    %% bash -c "t_dir=$t_dir"; "arch=$arch"; "hyp_st=$hyp_st"; "cv_wgt=$cv_wgt"; "out=$out"; 
    !echo training $out
    !python train_sealnet.py $t_dir $arch $hyp_st $cv_wgt $out
    
      

ERROR:root:Line magic function `%` not found.


training model1.tar
Traceback (most recent call last):
  File "train_sealnet.py", line 15, in <module>
    from train_sealnet_nas_scalable import NASNetALarge
ModuleNotFoundError: No module named 'train_sealnet_nas_scalable'


ERROR:root:Line magic function `%` not found.


training model2.tar
Traceback (most recent call last):
  File "train_sealnet.py", line 15, in <module>
    from train_sealnet_nas_scalable import NASNetALarge
ModuleNotFoundError: No module named 'train_sealnet_nas_scalable'


ERROR:root:Line magic function `%` not found.


...

training model6.tar
Traceback (most recent call last):
  File "train_sealnet.py", line 15, in <module>
    from train_sealnet_nas_scalable import NASNetALarge
ModuleNotFoundError: No module named 'train_sealnet_nas_scalable'


ERROR:root:Line magic function `%` not found.


training model7.tar
Traceback (most recent call last):
  File "train_sealnet.py", line 15, in <module>
    from train_sealnet_nas_scalable import NASNetALarge
ModuleNotFoundError: No module named 'train_sealnet_nas_scalable'


ERROR:root:Line magic function `%` not found.


training model8.tar
Traceback (most recent call last):
  File "train_sealnet.py", line 15, in <module>
    from train_sealnet_nas_scalable import NASNetALarge
ModuleNotFoundError: No module named 'train_sealnet_nas_scalable'


## Validation